In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
!unzip -q ./gdrive/My\ Drive/DP_punctuator/DP_final_exams_autoresolver.zip

In [0]:
!unzip -q ./gdrive/My\ Drive/DP_punctuator/dataset.zip

In [0]:
%%capture
!pip install -r requirements.txt

In [0]:
!mkdir data

In [0]:
import json
import re
import attr
import logging
import numpy as np
import os
import shutil
import torch
import torch.nn as nn
from operator import attrgetter


logger = logging.getLogger(__name__)


@attr.s(frozen=True)
class Field(object):
    name = attr.ib()
    dtype = attr.ib()


def save_model(model, optimizer, scheduler, model_path):
    model_dir = os.path.dirname(model_path)
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }, model_path + '_tmp')
    shutil.copyfile(model_path + '_tmp', model_path)
    os.remove(model_path + '_tmp')


def load_tasks(dir_path, task_num=None):
    if not isinstance(task_num, list):
        task_num = [task_num]
    tasks, filenames = [], [os.path.join(dir_path, f) for f in os.listdir(dir_path)]
    for filename in filenames:
        if filename.endswith(".json"):
            with open(filename, encoding='utf-8') as f:
                dt = f.read().encode('utf-8')
                data = json.loads(dt)
                if not isinstance(data, list):
                    data = [data]
                tasks += [d for d in data if 'id' in d and int(d['id']) in task_num]
    return tasks


def fix_spaces(text):
    return re.compile('\s+').sub(' ', text)


class BatchCollector(object):
    def __init__(self, matrix_fields, vector_fields, device):
        self._matrix_fields = matrix_fields
        self._vector_fields = vector_fields
        self._device = device

    def __call__(self, samples):
        def batchify_matrix(get_field, dtype):
            tensor = np.zeros((len(samples), max_length), dtype=dtype)

            for sample_id, sample in enumerate(samples):
                data = get_field(sample)
                tensor[sample_id, :len(data)] = data

            return torch.from_numpy(tensor)

        def batchify_vector(get_field, dtype):
            return torch.as_tensor([get_field(sample) for sample in samples], dtype=dtype)

        max_length = max(len(sample) for sample in samples)

        batch = {
            field.name: batchify_matrix(attrgetter(field.name), field.dtype).to(self._device)
            for field in self._matrix_fields
        }
        for field in self._vector_fields:
            batch[field.name] = batchify_vector(attrgetter(field.name), field.dtype).to(self._device)

        return batch


class AccuracyCounter(object):
    def __init__(self, name=None, masked_values=None):
        self.name = name
        self.correct_count = 0.
        self.total_count = 0.
        self._masked_values = masked_values or []

    def update(self, predictions, labels):
        mask = torch.ones_like(labels, dtype=torch.bool)
        for masked_value in self._masked_values:
            mask &= labels != masked_value

        self.correct_count += ((predictions == labels).float() * mask.float()).sum()
        self.total_count += mask.sum()

    @property
    def value(self):
        return self.correct_count / self.total_count

    def __str__(self):
        prefix = '{}: '.format(self.name) if self.name else 'Acc: '
        return prefix + '{:.2%}'.format(self.value)


class ConsoleProgressBar(object):
    def __init__(self, total):
        self._step = 0
        self._total_steps = total

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        pass

    def update(self):
        self._step += 1

    def set_description(self, text):
        if self._step % 100 == 0:
            logger.info(text + ' | [{}/{}]'.format(self._step, self._total_steps))

    def refresh(self):
        pass


class ModelTrainer(object):
    def __init__(self, model, optimizer, scheduler=None, use_tqdm=False, model_path=None, clip_norm=5.):
        self._model = model
        self._optimizer = optimizer
        self._scheduler = scheduler
        self._use_tqdm = use_tqdm
        self._model_path = model_path
        self._clip_norm = clip_norm
        self._global_step = 0

    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self._epoch_loss = 0.
        self._is_train = is_train
        self._name = name
        self._batches_count = batches_count

        self._model.train(is_train)

    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}'.format(
            self._name, self._epoch_loss / self._batches_count
        )

    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """

        loss, metrics_info = self.forward_pass(batch)
        self._epoch_loss += loss.item()

        if self._is_train:
            self.backward_pass(loss)
            self._global_step += 1

            if self._global_step % 1000 == 0 and self._model_path:
                save_model(self._model, self._optimizer, self._scheduler, self._model_path)

        return '{:>5s} Loss = {:.5f}, '.format(self._name, loss.item()) + metrics_info

    def forward_pass(self, batch):
        outputs = self._model(batch)
        return outputs['loss'], 'Loss = {:.3f}'.format(outputs['loss'].item())

    def backward_pass(self, loss):
        self._optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self._model.parameters(), self._clip_norm)
        self._optimizer.step()
        if self._scheduler:
            self._scheduler.step()

    def fit(self, train_iter, train_batches_per_epoch=None, epochs_count=1,
            val_iter=None, val_batches_per_epoch=None):

        train_batches_per_epoch = train_batches_per_epoch or len(train_iter)
        if val_iter is not None:
            val_batches_per_epoch = val_batches_per_epoch or len(val_iter)

        try:
            for epoch in range(epochs_count):
                name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
                self._do_epoch(iter(train_iter), is_train=True,
                            batches_count=train_batches_per_epoch,
                            name=name_prefix + 'Train:')

                if val_iter is not None:
                    self._do_epoch(iter(val_iter), is_train=False,
                                batches_count=val_batches_per_epoch,
                                name=name_prefix + '  Val:')
        except KeyboardInterrupt:
            logger.info('Early stopping was triggered')

        if self._model_path:
            save_model(self._model, self._optimizer, self._scheduler, self._model_path)

    def _do_epoch(self, data_iter, is_train, batches_count, name=None):
        self.on_epoch_begin(is_train, name, batches_count=batches_count)

        progress_bar_class = ConsoleProgressBar
        if self._use_tqdm:
            try:
                from tqdm import tqdm
                tqdm.get_lock().locks = []

                progress_bar_class = tqdm
            except:
                pass

        with torch.autograd.set_grad_enabled(is_train):
            with progress_bar_class(total=batches_count) as progress_bar:
                try:
                    for _ in range(batches_count):
                        batch = next(data_iter)
                        batch_progress = self.on_batch(batch)

                        progress_bar.update()
                        progress_bar.set_description(batch_progress)
                except StopIteration:
                    pass
                epoch_progress = self.on_epoch_end()
                progress_bar.set_description(epoch_progress)
                progress_bar.refresh()


In [0]:
import re
import numpy as np
import attr
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import transformers
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

CLS_TOKEN = '[CLS]'
SEP_TOKEN = '[SEP]'
MASK_TOKEN = '[MASK]'


@attr.s(frozen=True)
class Example(object):
    tokens = attr.ib()
    token_ids = attr.ib()
    segment_ids = attr.ib()
    mask = attr.ib()
    position = attr.ib()
    label_id = attr.ib(default=None)

    def __len__(self):
        return len(self.token_ids)


@attr.s(frozen=True)
class TrainConfig(object):
    learning_rate = attr.ib(default=1e-5)
    train_batch_size = attr.ib(default=32)
    test_batch_size = attr.ib(default=32)
    epoch_count = attr.ib(default=25)
    warm_up = attr.ib(default=0.1)


def _get_optimizer(model, train_size, config):
    num_total_steps = int(train_size / config.train_batch_size * config.epoch_count)
    num_warmup_steps = int(num_total_steps * config.warm_up)

    optimizer = AdamW(model.parameters(), lr=config.learning_rate, correct_bias=False)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_total_steps)

    return optimizer, scheduler


def _get_batch_collector(device, is_train=True):
    if is_train:
        vector_fields = [Field('label_id', torch.float32)]
    else:
        vector_fields = []

    vector_fields.append(Field('position', torch.long))

    return BatchCollector(
        matrix_fields=[
            Field('token_ids', np.int64),
            Field('segment_ids', np.int64),
            Field('mask', np.int64),
        ],
        vector_fields=vector_fields,
        device=device
    )


class BertClassifier(nn.Module):
    def __init__(self, bert, output_name, bert_output_dim=768):
        super(BertClassifier, self).__init__()

        self._bert = bert
        self._dropout = nn.Dropout(0.3)
        self._predictor = nn.Linear(bert_output_dim, 1)
        self._output_name = output_name

    def forward(self, batch):
        outputs, pooled_outputs = self._bert(
            input_ids=batch['token_ids'],
            attention_mask=batch['mask'],
            token_type_ids=batch['segment_ids'],
        )

        row_positions = torch.arange(0, outputs.shape[0])
        outputs = outputs[row_positions, batch['position']]

        status_logits = self._predictor(self._dropout(outputs)).squeeze(-1)

        loss = 0.
        if self._output_name + '_id' in batch:
            loss = F.binary_cross_entropy_with_logits(
                status_logits, batch[self._output_name + '_id']
            )

        return {
            self._output_name + '_logits': status_logits,
            'loss': loss
        }


class ClassifierTrainer(ModelTrainer):
    def on_epoch_begin(self, *args, **kwargs):
        super(ClassifierTrainer, self).on_epoch_begin(*args, **kwargs)

        self._accuracies = [AccuracyCounter()]

    def on_epoch_end(self):
        info = super(ClassifierTrainer, self).on_epoch_end()

        return '{}, {}'.format(info, ', '.join(str(score) for score in self._accuracies))

    def forward_pass(self, batch):
        outputs = self._model(batch)

        predictions = (outputs['label_logits'] > 0.).float()
        assert predictions.shape == batch['label_id'].shape

        self._accuracies[0].update(predictions, batch['label_id'])

        info = ', '.join(str(score) for score in self._accuracies)
        return outputs['loss'], info


class Solver(object):
    def __init__(self, model_name):
        self._model_name = model_name
        self._tokenizer = transformers.AutoTokenizer.from_pretrained(self._model_name)
        self._bert = transformers.BertModel.from_pretrained(self._model_name)
        self._device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self._batch_collector = _get_batch_collector(self._device, is_train=False)
        self._model = BertClassifier(self._bert, output_name='label').to(self._device)

    @staticmethod
    def _get_sentences(text):
        for sent in text.split('.'):
            if re.search('\(\d+\)', sent):
                yield fix_spaces(sent.strip()) + '.'

    def _convert_task(self, task):
        text = task["text"].replace("?", ".").replace("\xa0", "")
        sentence = ' '.join(list(self._get_sentences(text)))

        correct = None
        if 'solution' in task:
            if 'correct' in task['solution']:
                solution = task['solution']['correct']
            else:
                solution = task['solution']['correct_variants'][0]
            correct = sorted([int(idx) - 1 for idx in solution])

        tokens, positions = [CLS_TOKEN], []
        prev_match_end = 0
        for match in re.finditer('\(\d+\)', sentence):
            tokens.extend(self._tokenizer.tokenize(sentence[prev_match_end: match.start()].strip()))
            positions.append(len(tokens))
            tokens.append(MASK_TOKEN)
            prev_match_end = match.end()

        tokens.extend(self._tokenizer.tokenize(sentence[prev_match_end:].strip()))
        tokens.append(SEP_TOKEN)
        assert all(tokens[position] == MASK_TOKEN for position in positions)

        token_ids = self._tokenizer.convert_tokens_to_ids(tokens)
        assert token_ids[0] == 101 and token_ids[-1] == 102
        assert len(token_ids) == len(tokens)

        for position_id, position in enumerate(positions):
            label_id = None
            if correct is not None:
                label_id = int(position_id in correct)

            yield Example(
                tokens=tokens,
                token_ids=token_ids,
                segment_ids=[0] * len(token_ids),
                mask=[1] * len(token_ids),
                position=position,
                label_id=label_id
            )

    def _prepare_examples(self, tasks):
        examples = []
        for task in tasks:
            for example in self._convert_task(task):
                examples.append(example)
        return examples

    def fit(self, tasks, test_tasks=None):
        train_examples = self._prepare_examples(tasks)
        test_examples = None
        if test_tasks is not None:
            test_examples = self._prepare_examples(test_tasks)

        config = TrainConfig()
        self._model = BertClassifier(
            transformers.BertModel.from_pretrained(self._model_name),
            output_name='label'
        ).to(self._device)

        batch_collector = _get_batch_collector(self._device, is_train=True)

        train_loader = DataLoader(
            train_examples, batch_size=config.train_batch_size,
            shuffle=True, collate_fn=batch_collector, pin_memory=False
        )
        test_loader, test_batches_per_epoch = None, 0
        if test_examples is not None:
            test_loader = DataLoader(
                test_examples, batch_size=config.test_batch_size,
                shuffle=False, collate_fn=batch_collector, pin_memory=False
            )
            test_batches_per_epoch = int(len(test_examples) / config.test_batch_size)

        optimizer, scheduler = _get_optimizer(self._model, len(train_examples), config)

        trainer = ClassifierTrainer(
            self._model, optimizer, scheduler, use_tqdm=True
        )
        trainer.fit(
            train_iter=train_loader,
            train_batches_per_epoch=int(len(train_examples) / config.train_batch_size),
            val_iter=test_loader, val_batches_per_epoch=test_batches_per_epoch,
            epochs_count=config.epoch_count
        )

    def save(self, path="data/model_solver17.pkl"):
        torch.save(self._model.state_dict(), path)

    def load(self, path="data/model_solver17.pkl"):
        model_checkpoint = torch.load(path, map_location=self._device)
        self._model.load_state_dict(model_checkpoint)
        self._model.eval()

    def predict_from_model(self, task):
        examples = list(self._convert_task(task))
        model_inputs = self._batch_collector(examples)

        with torch.no_grad():
            model_prediction = self._model(model_inputs)['label_logits'].cpu().numpy()

        choices = task['question']['choices']
        prediction = [
            str(choices[ind]['id'])
            for ind, logit in enumerate(model_prediction)
            if logit > 0.
        ]

        return prediction




In [0]:
SEED = 42

def seed_everything(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

Train different BERT models over all 17-20 tasks at the same time

In [0]:
model_names = [
  'DeepPavlov/rubert-base-cased',
  'DeepPavlov/rubert-base-cased-sentence',
  'DeepPavlov/bert-base-multilingual-cased-sentence',
  'bert-base-multilingual-cased',
  'bert-base-multilingual-uncased'
]

for model_name in model_names:
    print()
    print()
    print()
    print("------------------------")
    print("train ", model_name)
    print("------------------------")
    print()

    seed_everything(SEED)

    solver = Solver(model_name)

    train_tasks, test_tasks = [], []
    for task_id in range(17, 21):
        train_tasks.extend(load_tasks('dataset/train/', task_num=task_id))
        test_tasks.extend(load_tasks('dataset/test/', task_num=task_id))

    seed_everything(SEED)

    solver.fit(train_tasks, test_tasks)

    model_fn = "data/model_" + model_name.replace("/", "_") + "_all_task.pkl"

    solver.save(model_fn)

    # print()
    # print("predict on test datases")
    
    # solver = Solver(model_name)
    # solver.load(model_fn)
    # task = load_tasks('dataset/test/', task_num=17)[0]
    # print(task)
    # print(solver.predict_from_model(task))





------------------------
train  DeepPavlov/rubert-base-cased
------------------------



[25 / 25]   Val: Loss = 0.09999, Acc: 98.59%: 100%|██████████| 162/162 [00:10<00:00, 15.32it/s]





------------------------
train  DeepPavlov/rubert-base-cased-sentence
------------------------



[25 / 25]   Val: Loss = 0.09958, Acc: 98.57%: 100%|██████████| 162/162 [00:10<00:00, 16.17it/s]





------------------------
train  DeepPavlov/bert-base-multilingual-cased-sentence
------------------------



[25 / 25]   Val: Loss = 0.15057, Acc: 98.03%: 100%|██████████| 162/162 [00:14<00:00, 10.83it/s]





------------------------
train  bert-base-multilingual-cased
------------------------



[25 / 25]   Val: Loss = 0.10850, Acc: 98.50%: 100%|██████████| 162/162 [00:14<00:00, 10.84it/s]





------------------------
train  bert-base-multilingual-uncased
------------------------



[25 / 25]   Val: Loss = 0.10314, Acc: 98.61%: 100%|██████████| 162/162 [00:14<00:00, 11.14it/s]


Train different BERT models over 17-20 tasks separately

In [0]:
model_names = [
  'DeepPavlov/rubert-base-cased',
  'DeepPavlov/rubert-base-cased-sentence',
  'DeepPavlov/bert-base-multilingual-cased-sentence',
  'bert-base-multilingual-cased',
  'bert-base-multilingual-uncased'
]

def train_model(model_name):
    print()
    print()
    print()
    print("------------------------")
    print("train ", model_name)
    print("------------------------")
    print()

    for task_id in range(17, 21):
        print()
        print("------------------------")
        print("train task", task_id)
        print("------------------------")
        print()

        seed_everything(SEED)

        solver = Solver(model_name)

        train_tasks, test_tasks = [], []
        train_tasks.extend(load_tasks('dataset/train/', task_num=task_id))
        test_tasks.extend(load_tasks('dataset/test/', task_num=task_id))

        seed_everything(SEED)

        solver.fit(train_tasks, test_tasks)

        model_fn = "data/model_" + model_name.replace("/", "_") + "_task_{}.pkl".format(task_id)

        solver.save(model_fn)

        # print()
        # print("predict on test datases")
        
        # solver = Solver(model_name)
        # solver.load(model_fn)
        # task = load_tasks('dataset/test/', task_num=task_id)[0]
        # print(task)
        # print(solver.predict_from_model(task))


In [13]:
train_model('DeepPavlov/rubert-base-cased')




------------------------
train  DeepPavlov/rubert-base-cased
------------------------


------------------------
train task 17
------------------------



[25 / 25]   Val: Loss = 0.07071, Acc: 98.56%: 100%|██████████| 39/39 [00:02<00:00, 16.24it/s]



------------------------
train task 18
------------------------



[25 / 25]   Val: Loss = 0.22210, Acc: 96.47%: 100%|██████████| 46/46 [00:03<00:00, 12.36it/s]



------------------------
train task 19
------------------------



[25 / 25]   Val: Loss = 0.04462, Acc: 99.05%: 100%|██████████| 36/36 [00:02<00:00, 16.16it/s]



------------------------
train task 20
------------------------



[25 / 25]   Val: Loss = 0.04576, Acc: 99.06%: 100%|██████████| 40/40 [00:02<00:00, 17.08it/s]


In [14]:
train_model('DeepPavlov/rubert-base-cased-sentence')




------------------------
train  DeepPavlov/rubert-base-cased-sentence
------------------------


------------------------
train task 17
------------------------



[25 / 25]   Val: Loss = 0.08487, Acc: 98.48%: 100%|██████████| 39/39 [00:02<00:00, 17.37it/s]



------------------------
train task 18
------------------------



[25 / 25]   Val: Loss = 0.41965, Acc: 92.80%: 100%|██████████| 46/46 [00:03<00:00, 12.74it/s]



------------------------
train task 19
------------------------



[25 / 25]   Val: Loss = 0.05729, Acc: 99.05%: 100%|██████████| 36/36 [00:02<00:00, 17.66it/s]



------------------------
train task 20
------------------------



[25 / 25]   Val: Loss = 0.13236, Acc: 98.05%: 100%|██████████| 40/40 [00:02<00:00, 17.97it/s]


In [15]:
train_model('DeepPavlov/bert-base-multilingual-cased-sentence')




------------------------
train  DeepPavlov/bert-base-multilingual-cased-sentence
------------------------


------------------------
train task 17
------------------------



[25 / 25]   Val: Loss = 0.12275, Acc: 97.76%: 100%|██████████| 39/39 [00:03<00:00, 11.63it/s]



------------------------
train task 18
------------------------



[25 / 25]   Val: Loss = 0.44231, Acc: 92.53%: 100%|██████████| 46/46 [00:05<00:00,  9.08it/s]



------------------------
train task 19
------------------------



[25 / 25]   Val: Loss = 0.06791, Acc: 98.87%: 100%|██████████| 36/36 [00:03<00:00, 11.36it/s]



------------------------
train task 20
------------------------



[25 / 25]   Val: Loss = 0.11461, Acc: 98.28%: 100%|██████████| 40/40 [00:03<00:00, 11.63it/s]


In [16]:
train_model('bert-base-multilingual-cased')




------------------------
train  bert-base-multilingual-cased
------------------------


------------------------
train task 17
------------------------



[25 / 25]   Val: Loss = 0.10966, Acc: 98.16%: 100%|██████████| 39/39 [00:03<00:00, 11.69it/s]



------------------------
train task 18
------------------------



[25 / 25]   Val: Loss = 0.29351, Acc: 94.43%: 100%|██████████| 46/46 [00:05<00:00,  9.16it/s]



------------------------
train task 19
------------------------



[25 / 25]   Val: Loss = 0.05146, Acc: 99.13%: 100%|██████████| 36/36 [00:03<00:00, 11.43it/s]



------------------------
train task 20
------------------------



[25 / 25]   Val: Loss = 0.08444, Acc: 98.91%: 100%|██████████| 40/40 [00:03<00:00, 11.62it/s]


In [17]:
train_model('bert-base-multilingual-uncased')




------------------------
train  bert-base-multilingual-uncased
------------------------


------------------------
train task 17
------------------------



[25 / 25]   Val: Loss = 0.08516, Acc: 98.40%: 100%|██████████| 39/39 [00:03<00:00, 11.90it/s]



------------------------
train task 18
------------------------



[25 / 25]   Val: Loss = 0.28522, Acc: 95.45%: 100%|██████████| 46/46 [00:04<00:00,  9.50it/s]



------------------------
train task 19
------------------------



[25 / 25]   Val: Loss = 0.07265, Acc: 99.05%: 100%|██████████| 36/36 [00:03<00:00, 11.67it/s]



------------------------
train task 20
------------------------



[25 / 25]   Val: Loss = 0.08062, Acc: 98.83%: 100%|██████████| 40/40 [00:03<00:00, 12.04it/s]
